In [ ]:
# Convert 'date' and 'minute' to a datetime format that can be used in time series models
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['minute'].astype(str), format='%Y-%m-%d %H%M')

# Set datetime as the index
df.set_index('datetime', inplace=True)

Autoregressive Integrated Moving Average

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit an ARIMA model
arima_model = ARIMA(df['aapl_close'], order=(1, 1, 1))
arima_result = arima_model.fit()
print(arima_result.summary())


LSTM Model Implementation

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.optimizers import Adam

# Preparing data for LSTM
def create_sequences(data, window_size=10):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        sequences.append(data.iloc[i:(i + window_size)].values)
        labels.append(data.iloc[i + window_size])
    return np.array(sequences), np.array(labels)

window_size = 10  # number of past days to consider
X_train, y_train = create_sequences(train_data['Close'], window_size)
X_val, y_val = create_sequences(val_data['Close'], window_size)

# LSTM Model Architecture
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(window_size, 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(train_data[['moving_average_5', 'moving_average_10']], train_data['Close'])

# Prediction and Evaluation
predictions = rf_model.predict(val_data[['moving_average_5', 'moving_average_10']])
mse = mean_squared_error(val_data['Close'], predictions)
print(f'Mean Squared Error: {mse}')

# Feature Importance
importances = rf_model.feature_importances_
plt.barh(['moving_average_5', 'moving_average_10'], importances)
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()
